In [ ]:

import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


df = pd.read_excel("ML_ Data.xlsx", sheet_name="Sheet1")
df_clean = df.dropna()


df_clean = df_clean.rename(columns={
    "Feed (mm/rev)": "Feed",
    "Depth of cut [µm]": "Depth_of_cut",
    "h (Average)": "h",
    "T_Model": "T_Analytical"
})


y = df_clean["T_Exp"]

# ------------------ NON-PIML MODEL ------------------
X_non_piml = df_clean[["Feed", "Depth_of_cut", "Fx", "Fy"]]
X_train_non, X_test_non, y_train, y_test = train_test_split(X_non_piml, y, test_size=0.2, random_state=42)

xgb_non = XGBRegressor(random_state=42)
xgb_non.fit(X_train_non, y_train)
y_pred_non = xgb_non.predict(X_test_non)


r2_non = r2_score(y_test, y_pred_non)
rmse_non = np.sqrt(mean_squared_error(y_test, y_pred_non))  
mae_non = mean_absolute_error(y_test, y_pred_non)
mape_non = mean_absolute_percentage_error(y_test, y_pred_non) * 100

# ------------------ PIML MODEL ------------------
X_piml = df_clean[["Feed", "Depth_of_cut", "Fx", "Fy", "T_Analytical"]]  # remove h
X_train_piml, X_test_piml, _, _ = train_test_split(X_piml, y, test_size=0.2, random_state=42)

xgb_piml = XGBRegressor(random_state=42)
xgb_piml.fit(X_train_piml, y_train)
y_pred_piml = xgb_piml.predict(X_test_piml)


r2_piml = r2_score(y_test, y_pred_piml)
rmse_piml = np.sqrt(mean_squared_error(y_test, y_pred_piml))
mae_piml = mean_absolute_error(y_test, y_pred_piml)
mape_piml = mean_absolute_percentage_error(y_test, y_pred_piml) * 100


results = pd.DataFrame({
    "Model": ["Non-PIML", "PIML"],
    "R²": [r2_non, r2_piml],
    "RMSE": [rmse_non, rmse_piml],
    "MAE": [mae_non, mae_piml],
    "MAPE (%)": [mape_non, mape_piml]
})

print("Model Performans Karşılaştırması:")
print(results)


results_non = pd.DataFrame({"True": y_test.values, "Predicted (Non-PIML)": y_pred_non})
results_piml = pd.DataFrame({"True": y_test.values, "Predicted (PIML)": y_pred_piml})


# results_non.to_excel("XGBoost_Non_PIML_Output.xlsx", index=False)
# results_piml.to_excel("XGBoost_PIML_Output.xlsx", index=False)
# results.to_excel("XGBoost_Performance_Comparison.xlsx", index=False)


Model Performans Karşılaştırması:
      Model        R²       RMSE        MAE   MAPE (%)
0  Non-PIML  0.132287  51.177903  43.233360  48.658147
1      PIML  0.109808  51.836577  44.060375  47.820175
